<a href="https://colab.research.google.com/github/oldaandozerskaya/SimilarBooks/blob/main/SimilarBooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests as req
import re

In [ ]:
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
url = "https://www.livelib.ru/book/1001445982-pobeg-iz-shoushenka-stiven-king"
resp = req.get(url)
http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
encoding = html_encoding or http_encoding
soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding)

In [ ]:
soup.title.text

In [ ]:
def check_similar(url):#возвращает список похожих книг
  similar_url = 'https://www.livelib.ru/book/'
  pos = url.find('-')
  similar_url += str(url[28:])+'/similar'
  resp = req.get(similar_url)
  http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
  html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
  encoding = html_encoding or http_encoding
  soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding)
  items = soup.find_all('a', class_='brow-book-name with-cycle')
  similar_books = []
  for item in items:
    pos = str(item).find('edition_id=')
    temp = str(item)[pos:]
    pos = temp.find('&')
    similar_books.append('https://www.livelib.ru/book/'+temp[11:pos])
  return similar_url, similar_books

#check_similar('https://www.livelib.ru/book/1001445982')

In [ ]:
def get_plot(url): #получить сюжет
  resp = req.get(url)
  http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
  html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
  encoding = html_encoding or http_encoding
  soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding)
  content = str(soup)
  pos = content.find('Сюжет</label>')
  if pos>0:
    temp = content[pos+13:]
    pos = temp.find('</div>')
    return temp[:pos]
  else:
    return ''

def clean_plot(text): #очистить текст от тэгов
  try:
    text = text.replace('\r\n',' ')
    text = text.replace('\n',' ')
    new_text = ''
    write = True
    for s in text:
      if s=='<':
        write = False
      if write:
        new_text += s
      if s=='>':
        write = True
    new_text = re.sub(' +', ' ', new_text)
    if new_text[0]==' ':
      new_text=new_text[1:]
    return new_text.replace('  ',' ')
  except:
    return ''

#url = "https://www.livelib.ru/book/1001445982-pobeg-iz-shoushenka-stiven-king"
#print(clean_plot(get_plot(url)))

In [ ]:
def get_abstract(url): #получить аннотацию
  resp = req.get(url)
  http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
  html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
  encoding = html_encoding or http_encoding
  soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding)
  content = str(soup)
  pos = content.find('<p itemprop="description">')
  if pos>0:
    temp = content[pos+26:]
    pos = temp.find('</div>')
    return temp[:pos]
  else:
    return ''

#print(clean_plot(get_abstract(url)))

In [ ]:
#получить список ссылок на книги со страницы

url = 'https://www.livelib.ru/genre/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F-%D0%BB%D0%B8%D1%82%D0%B5%D1%80%D0%B0%D1%82%D1%83%D1%80%D0%B0'


def get_books(url):
  resp = req.get(url)
  http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
  html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
  encoding = html_encoding or http_encoding
  soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding)

  content = str(soup)
  st = '<a class="brow-expand ll-toggle-text nowrap" href="javascript:void(0);" data-id="book-'
  l = len(st)

  pos = 0
  ids = []

  items = soup.find_all("a", class_='brow-expand ll-toggle-text nowrap')

  for item in items:
    num = str(item)[59:69]
    ids.append('https://www.livelib.ru/book/'+num)

  return ids

#get_books(url)

In [ ]:
#получить атрибуты книги
def get_attr(url):
  resp = req.get(url)
  http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
  html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
  encoding = html_encoding or http_encoding
  soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding)
  author = soup.find('span', class_='hidden').text
  title = soup.title.text
  genre = soup.find('div', class_='bc-genre__limiter').find_all('a')
  genres = []
  for i in range(len(genre)):
    pos = str(genre[i])[16:].find('/')
    item = str(genre[i])[16:pos+16]
    if item.find('Еще')==-1:
      genres.append(item)
  return author, title, genres

#get_attr(get_books(url)[0])

In [ ]:
def get_book_info(url):
  resp = req.get(url)
  http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
  html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
  encoding = html_encoding or http_encoding
  soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding)

  author = soup.find('span', class_='hidden').text
  title = soup.title.text
  genre = soup.find('div', class_='bc-genre__limiter').find_all('a')
  genres = []
  for i in range(len(genre)):
    pos = str(genre[i])[16:].find('/')
    item = str(genre[i])[16:pos+16]
    if item.find('Еще')==-1:
      genres.append(item)

  content = str(soup)
  pos = content.find('<p itemprop="description">')
  if pos>0:
    temp = content[pos+26:]
    pos = temp.find('</div>')
    abstract = temp[:pos]
  else:
    abstract = ''

  pos = content.find('Сюжет</label>')
  if pos>0:
    temp = content[pos+13:]
    pos = temp.find('</div>')
    plot = temp[:pos]
  else:
    plot = ''

  return author, title, genres, clean_plot(abstract), clean_plot(plot)

In [ ]:
import csv
classic_url = 'https://www.livelib.ru/genre/Зарубежная-литература/best/listview/biglist/~'
num = 1
max_num = 50
import time
import random

book_id = 230038
#fields = ['id', 'title1', 'author1', 'genres1', 'title2', 'author2', 'genres2']
#with open('/content/drive/My Drive/SimilarBooks/pairs.csv', 'a') as f:
    #writer = csv.writer(f)
    #writer.writerow(fields)

similar_books = []
book_name = 'Жареные зеленые помидоры в кафе «Полустанок»'
started = False

for i in range(num, max_num):
  print('===============================================')
  print(i)
  time.sleep(random.randint(5,10))
  books = get_books(classic_url+str(i))
  
  for book_url in books:
    time.sleep(random.randint(3,20))
    similar_url, similar_books = check_similar(book_url)
    if str(similar_books) == '[]':
      continue
    else:
      time.sleep(random.randint(5,10))
      a1, t1, g1, abstract1, plot1 = get_book_info(book_url)
      
      if t1==book_name:
        started = True
        continue
      if started:
        if plot1 == '':
          continue
        else:
          for s_book in similar_books:
              time.sleep(random.randint(3,20))
              try:
                a2, t2, g2, abstract2, plot2 = get_book_info(s_book)
              except:
                continue
              if plot2 == '':
                continue
              else:
                print(t1)
                print(t2) 
                print('***')  
                fields = [book_id, t1, a1, g1, t2, a2, g2]
                with open('/content/drive/My Drive/SimilarBooks/pairs.csv', 'a') as f:
                  writer = csv.writer(f)
                  writer.writerow(fields)

                with open('/content/drive/My Drive/SimilarBooks/plots/'+str(book_id)+'_1.txt',\
                          encoding='utf-8', mode = 'w') as f:
                  f.write(plot1)
                
                with open('/content/drive/My Drive/SimilarBooks/plots/'+str(book_id)+'_2.txt',\
                          encoding='utf-8', mode = 'w') as f:
                  f.write(plot2)
                
                with open('/content/drive/My Drive/SimilarBooks/abstracts/'+str(book_id)+'_1.txt',\
                          encoding='utf-8', mode = 'w') as f:
                  f.write(abstract1)
                
                with open('/content/drive/My Drive/SimilarBooks/abstracts/'+str(book_id)+'_2.txt', \
                          encoding='utf-8', mode = 'w') as f:
                  f.write(abstract2)

                book_id+=1 
          


In [ ]:
from google.colab import drive
drive.mount('/content/drive')